In [1]:
import torch
torch.cuda.is_available()

True

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
X_train = torch.FloatTensor([0., 1., 2.])
X_train.is_cuda

False

In [6]:
X_train = X_train.to(device)
X_train.is_cuda

True

In [8]:
X_train= torch.FloatTensor([0., 1., 2.])
X_train = X_train.cuda()
X_train.is_cuda

True

In [13]:
X_train= torch.FloatTensor([0., 1., 2.])
X_train = X_train.cpu()
X_train.is_cuda

False

In [14]:
X_train= torch.FloatTensor([0., 1., 2.])
X_train.is_cuda

False

In [12]:
torch.cuda.current_device()

0

In [10]:
torch.cuda.device_count()

1

In [11]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650'